In [1]:
# import nltk
# nltk.download('wordnet')
! pip install --upgrade mlflow

In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df= pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv',index=False)
df.head()


,review,sentiment
590,"Twenty five years ago, I showed this film in s...",positive
184,"Narratives  whether written, visual or poetic...",positive
220,I saw this movie in Blockbuster and thought it...,negative
94,I rented this film on Netflix after it won all...,negative
60,What could've been a great film about the late...,negative


In [4]:
#  data preprocessing 
#  Define test preprocessing function
def lammatization(text):
    """Lamatize the text"""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text"""
    stop_words = set(stopwords.words('english'))
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

def remove_number(text):
    """Remove numbers from the text"""
    text = ''.join([char for char in text if not char.isdigit()])
    return text
    
def lower_case(text):
    """Convert the text to lower case"""
    text=text.split()
    text=[word.lower() for word in text]
    return ' '.join(text)

def removing_punctuation(text):
    '''Remove punctuations from the text'''
    text=re.sub('[%s]' % re.escape(string.punctuation),' ',text)
    text=text.replace(':',"")
    text=re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    '''Remove URLs from the text'''
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

In [5]:
def normalize_text(df):
    'Normalize the text data'
    try:
        df['review']=df['review'].apply(lower_case)
        df['review']=df['review'].apply(remove_stop_words)
        df['review']=df['review'].apply(removing_urls)
        df['review']=df['review'].apply(remove_number)
        df['review']=df['review'].apply(removing_punctuation)
        df['review']=df['review'].apply(lammatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise
        

In [6]:
df=normalize_text(df)
df.head()

,review,sentiment
590,twenty five year ago showed film child s class...,positive
184,narrative  whether written visual poetic epic...,positive
220,saw movie blockbuster thought might good sunda...,negative
94,rented film netflix oscar see really good br b...,negative
60,could ve great film late poker pro pre poker c...,negative


In [7]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [8]:
x= df['sentiment'].isin(['positive','negative'])
df=df[x]

In [9]:
df['sentiment']=df['sentiment'].map({'positive':1,'negative':0})
df.head()

,review,sentiment
590,twenty five year ago showed film child s class...,1
184,narrative  whether written visual poetic epic...,1
220,saw movie blockbuster thought might good sunda...,0
94,rented film netflix oscar see really good br b...,0
60,could ve great film late poker pro pre poker c...,0


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer=CountVectorizer(max_features= 100)

X=vectorizer.fit_transform(df['review'])
y=df['sentiment']



In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [17]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/TechGovinda/Review_Sentiment_Analysis.mlflow')
dagshub.init(repo_owner='TechGovinda', repo_name='Review_Sentiment_Analysis', mlflow=True)

# mlfloow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")



Initialized MLflow to track repo "TechGovinda/Review_Sentiment_Analysis"

2025-03-22 15:38:27,542 - INFO - Initialized MLflow to track repo "TechGovinda/Review_Sentiment_Analysis"


Repository TechGovinda/Review_Sentiment_Analysis initialized!

2025-03-22 15:38:27,545 - INFO - Repository TechGovinda/Review_Sentiment_Analysis initialized!


<Experiment: artifact_location='mlflow-artifacts:/5ff0693d58c2484eb2c81b8786a3bf33', creation_time=1742627268194, experiment_id='0', last_update_time=1742627268194, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [18]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-03-22 15:38:28,828 - INFO - Starting MLflow run...
2025-03-22 15:38:29,260 - INFO - Logging preprocessing parameters...
2025-03-22 15:38:30,452 - INFO - Initializing Logistic Regression model...
2025-03-22 15:38:30,458 - INFO - Fitting the model...
2025-03-22 15:38:30,550 - INFO - Model training complete.
2025-03-22 15:38:30,551 - INFO - Logging model parameters...
2025-03-22 15:38:30,930 - INFO - Making predictions...
2025-03-22 15:38:30,937 - INFO - Calculating evaluation metrics...
2025-03-22 15:38:30,964 - INFO - Logging evaluation metrics...
2025-03-22 15:38:32,598 - INFO - Saving and logging the model...
2025/03/22 15:38:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-22 15:38:42,646 - INFO - Model training and logging completed in 13.39 seconds.
2025-03-22 15:38:42,648 - INFO - Accuracy: 0.6
2025-03-22 15:38:42,649 - INFO - Precision: 0.

🏃 View run bold-koi-164 at: https://dagshub.com/TechGovinda/Review_Sentiment_Analysis.mlflow/#/experiments/0/runs/a3f6884affb34c57bad1b8c44646bd70
🧪 View experiment at: https://dagshub.com/TechGovinda/Review_Sentiment_Analysis.mlflow/#/experiments/0
